In [20]:
!ls  /home/jovyan/work/

aclImdb		  model.png	    plottingnetworks.ipynb  trytoghether.ipynb
angela.ipynb	  Nathantest.ipynb  Testmodel		    Webscraping.ipynb
angelatry2.ipynb  notebook.ipynb    try3.ipynb
init.ipynb	  out5.csv	    try4.ipynb
model_3.png	  out5.zip	    try5.ipynb


In [1]:
import csv
from pathlib import Path 
import numpy as np
from keras.preprocessing.text import Tokenizer
from sklearn.utils import shuffle
from keras.preprocessing import sequence
import os 

import tensorflow as tf

from keras import backend
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding, Reshape
from keras.layers import LSTM
from keras.layers import Conv1D, MaxPooling1D
from keras.utils import np_utils

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline


Using TensorFlow backend.


In [11]:
max_features = 20000
maxlen = 100





Ytrain=[]
trainx=[]
Ytest=[]
testx=[]
print("loading the data...")
with open('/home/jovyan/work/out7.csv',encoding="utf8") as csv_file:
    csv_reader = csv.reader(csv_file)
    for row in csv_reader:
        if csv_reader.line_num < 55001:
            Ytrain.append(int(row[0].split('/')[0]))
            trainx.append(row[0].split('\t')[1])
        elif csv_reader.line_num < 59300:
            Ytest.append(int(row[0].split('/')[0]))
            testx.append(row[0].split('\t')[1]) 
        else:
            break    


   

#59378


print("data loaded, processing data...")


# create the tokenizer
tok= Tokenizer(num_words=max_features, lower=True,filters='"#$%&()*+-/:<=>@[\\]^_`{|}~\t\n',split=' ', char_level=False, oov_token=None, document_count=0)
# fit the tokenizer on the documents
tok.fit_on_texts(trainx)
# summarize what was learned
#print(train.word_counts)
#print(train.document_count)
#print(train.word_index)
#print(train.word_docs)
# integer encode documents
Xtrain = tok.texts_to_sequences(trainx)

Xtest = tok.texts_to_sequences(testx)




Xtrain, y_train = shuffle(Xtrain, Ytrain, random_state=0)
Xtest, y_test = shuffle(Xtest, Ytest, random_state=0)


# encode class values as integers
encoder = LabelEncoder()
encoder.fit(Ytrain)
encoded_Ytrain = encoder.transform(Ytrain)
encoded_Ytest = encoder.transform(Ytest)
# convert integers to dummy variables (i.e. one hot encoded)
y_train = np_utils.to_categorical(encoded_Ytrain)
y_test = np_utils.to_categorical(encoded_Ytest)

encoder_length = len(y_train[0])
print("length of one hot enconding  ", encoder_length)

print('Pad sequences (samples x time)')
x_train = sequence.pad_sequences(Xtrain, maxlen=maxlen)
x_test = sequence.pad_sequences(Xtest, maxlen=maxlen)

print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

Ytrain = [x-1 for x in Ytrain]
Ytest = [x-1 for x in Ytest]



loading the data...


FileNotFoundError: [Errno 2] No such file or directory: '/home/jovyan/work/out7.csv'

In [0]:
# https://towardsdatascience.com/multi-class-text-classification-with-lstm-1590bee1bd17

from keras.layers import SpatialDropout1D
from keras.callbacks import EarlyStopping
from keras.metrics import top_k_categorical_accuracy


embedding_size = 100


model = Sequential()
model.add(Embedding(max_features,embedding_size, input_length=maxlen))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(encoder_length, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

epochs = 3
batch_size = 64

#Ytrain = [x-1 for x in Ytrain]
#Ytest = [x-1 for x in Ytest]


history2 = model.fit(x_train, Ytrain, epochs=epochs, batch_size=batch_size,validation_split=0.1,callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])



score, acc = model.evaluate(x_test, Ytest, batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)


/opt/venv/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
Train on 48004 samples, validate on 5334 samples
Epoch 1/3
21824/48004 [============>.................] - ETA: 3:12 - loss: 2.0130 - accuracy: 0.2964

48004/48004 [==============================] - 347s 7ms/step - loss: 2.0005 - accuracy: 0.2993 - val_loss: 2.0804 - val_accuracy: 0.2527
Epoch 2/3
48004/48004 [==============================] - 306s 6ms/step - loss: 1.9673 - accuracy: 0.3016 - val_loss: 2.0995 - val_accuracy: 0.2493
Epoch 3/3
4161/4161 [==============================] - 6s 1ms/step
Test score: 2.1662018654925754
Test accuracy: 0.23311704397201538


In [6]:
# https://towardsdatascience.com/multi-class-text-classification-with-lstm-1590bee1bd17

#with the shuffled data

from keras.layers import SpatialDropout1D
from keras.callbacks import EarlyStopping
from keras.metrics import top_k_categorical_accuracy


embedding_size = 100


model = Sequential()
model.add(Embedding(max_features,embedding_size, input_length=maxlen))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(encoder_length, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

epochs = 3
batch_size = 64

#Ytrain = [x-1 for x in Ytrain]
#Ytest = [x-1 for x in Ytest]


history2 = model.fit(x_train, Ytrain, epochs=epochs, batch_size=batch_size,validation_split=0.1,callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])



score, acc = model.evaluate(x_test, Ytest, batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)


/opt/venv/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
Train on 48004 samples, validate on 5334 samples
Epoch 1/3
48004/48004 [==============================] - 313s 7ms/step - loss: 1.9996 - accuracy: 0.2993 - val_loss: 2.0758 - val_accuracy: 0.2527
Epoch 2/3
48004/48004 [==============================] - 306s 6ms/step - loss: 1.9687 - accuracy: 0.3020 - val_loss: 2.1035 - val_accuracy: 0.2460
Epoch 3/3
4161/4161 [==============================] - 8s 2ms/step
Test score: 2.164063237544809
Test accuracy: 0.24321076273918152
